# RAG-based Question Answering

1. Set up the QA environment:
- Install OLLAMA and select an appropriate LLM
- Configure Qdrant vector database (or vector DB of your choosing)
- Install necessary Python packages for embedding generation

In [1]:
# pip install qdrant_client

In [2]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://6e2a612c-6d16-4ce6-8caa-94ae8febc9ea.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key="blQpESfpU10XfhMsGlTk8Vp7_d-tTV2hJgVg5z-85NRVZo5b67X1vQ",
)

print(qdrant_client.get_collections())


collections=[CollectionDescription(name='pdf_embeddings')]


In [3]:
# pip install sentence_transformers

In [4]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

2. Find PDF file of your choosing. Example - some publication or CV file.

3. Write next procedures necessary for RAG pipeline. Use LangChain library:
- Load PDF file using PyPDFLoader.
- Split documents into appropriate chunks using RecursiveCharacterTextSplitter.
- Generate and store embeddings in Qdrant database

In [5]:
# pip install langchain

In [6]:
# pip install -U langchain-community

In [7]:
# pip install pypdf

In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from qdrant_client.models import VectorParams

# Step 1: Load PDF file
pdf_loader = PyPDFLoader("example.pdf") 
documents = pdf_loader.load()

# Step 2: Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Maximum size of each chunk
    chunk_overlap=50,  # Overlap to retain context between chunks
)
chunks = text_splitter.split_documents(documents)

# Step 3: Generate embeddings
embeddings_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
texts = [chunk.page_content for chunk in chunks]
embeddings = embeddings_model.embed_documents(texts)

# Create a collection in Qdrant
collection_name = "pdf_embeddings"
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=len(embeddings[0]), distance="Cosine"),
)

# Step 5: Store embeddings in Qdrant
qdrant_client.upload_collection(
    collection_name=collection_name,
    vectors=embeddings,
    payload=[{"text": text} for text in texts],
    ids=list(range(len(embeddings))),
)

print(f"Stored {len(embeddings)} embeddings in Qdrant collection '{collection_name}'.")


C:\Users\gboch\AppData\Local\Temp\ipykernel_21524\3224759436.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\gboch\AppData\Local\Temp\ipykernel_21524\3224759436.py:24: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Stored 4 embeddings in Qdrant collection 'pdf_embeddings'.


4. Design and implement the RAG pipeline with LCEL. As reference use this detailed guide created by LangChain community - RAG. Next steps should involve:
- Create query embedding generation
- Implement semantic search in Qdrant
- Design prompt templates for context integration
- Build response generation with the LLM

5. Implement basic retrieval strategies (semantic search).

In [9]:
# pip install openai

In [10]:
# pip install tiktoken

In [11]:
def generate_query_embedding(query: str, embedding_model):
    return embedding_model.encode(query)

def semantic_search(query_embedding, collection_name: str, top_k: int = 5):
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k
    )
    return [
        {"text": result.payload["text"], "score": result.score}
        for result in search_results
    ]


6. Create basic QA prompt.

In [12]:
from langchain.prompts import PromptTemplate

template = """
Create a sentence based on the context {context}
and quetion {question}. Imagine you are a helpful
assistant"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

7. Determine 5 evaluation queries:
- Determine a few questions, which answers are confirmed by you.

In [13]:
import ollama

def generate_response(context: str, question: str, model_name='phi3:3.8b'):
    formatted_prompt = prompt.format(context=context, question=question)
    
    response = ollama.chat(
        model=model_name,
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )
    print(response)
    return response['message']['content']

def rag_pipeline(query, collection_name, embedding_model):
    query_embedding = generate_query_embedding(query, embedding_model)
    
    search_results = semantic_search(query_embedding, collection_name)
    
    context = "\n".join([result["text"] for result in search_results])
    
    response = generate_response(context, query)
    return response

new_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


In [14]:
evaluation_queries = [
    "What data analysis projects have you done in the past?",
    "What technologies and tools have you used most often in your work?",
    "Can you describe your experience working with SQL databases?",
    "What programming experience do you have?",
    "What do you consider the most important achievement in your career?"
]


query = evaluation_queries[0]
response = rag_pipeline(query, collection_name, new_embedding_model)
print("Odpowiedź na pytanie:", response)

model='phi3:3.8b' created_at='2025-01-17T19:37:36.4663234Z' done=True done_reason='stop' total_duration=411925634000 load_duration=16795923200 prompt_eval_count=739 prompt_eval_duration=92260000000 eval_count=678 eval_duration=302807000000 message=Message(role='assistant', content='I successfully completed several important data science and automation-related project, which I am proud to share:\n\n1. **Nurses Schedule, 2022/2023** - In this Python application developed during my studies at AGH University of Science and Technology in Poland (Edukacja od 2024 wraz z Bachelorem i Magisterem), I utilized machine learning algorithms to automate the weekly nurse scheduling process. The goal was to create an efficient system that minimizes conflicts while ensuring fair workload distribution among nursing staff, ultimately leading to increased job satisfaction and improved patient care within our healthcare facility in New Sącz, Poland (Adres: Gabriela Bocheńska).\n\n2. **Engineering Thesis, 2

In [15]:
query1 = evaluation_queries[1]
response = rag_pipeline(query1, collection_name, new_embedding_model)
print("Odpowiedź na pytanie:", response)

model='phi3:3.8b' created_at='2025-01-17T19:41:51.0655673Z' done=True done_reason='stop' total_duration=252771343600 load_duration=221171600 prompt_eval_count=742 prompt_eval_duration=71444000000 eval_count=882 eval_duration=181079000000 message=Message(role='assistant', content='Ale, w pracy odbiorę częstotliwość używania SQL, Pythona i narzędzia związane z inteljentem i uczeniem maszynowym. Stosujemy technologie jak Azure Machine Learning do tworzenia modeli predykcyjne, jak również system widzenia jakości (QA) dla analizowania danych oraz implementując automatyzację wdrożeń według Jenkinsa. Z drugiej strony, skryptów testowych stosujem Python, takie samo jak w celu przeprowadzania testów na platformie Azure App Service i WEB API. Dlatego, zobaczysz, że technologie centralne, które używam najczęściej to SQL oraz Pythona.\n\nPorzucone jest jednostki języka, ale dziś w pracy masz możliwość wykonywać te same działania językiem angielskim. W celu rozbudowa kariery na kierunku analiz dany

In [16]:
query2 = evaluation_queries[2]
response = rag_pipeline(query2, collection_name, new_embedding_model)
print("Odpowiedź na pytanie:", response)

model='phi3:3.8b' created_at='2025-01-17T19:43:16.7500748Z' done=True done_reason='stop' total_duration=84276019400 load_duration=91880100 prompt_eval_count=738 prompt_eval_duration=57931000000 eval_count=153 eval_duration=26220000000 message=Message(role='assistant', content="I have gained hands-on experience in managing and querying relational data using Structured Query Language (SQL). During my studies, I honed my skills through practical projects that involved extracting meaningful insights from large datasets stored within SQL databases. My proficiency extends to designing complex queries for aggregating information, joining multiple tables effectively, and ensuring the integrity of database transactions via appropriate transaction management practices such as 'COMMIT' in PostgreSQL or using ACID principles with SQLite when needed. Working directly on relational data has been instrumental not only in understanding fundamental concepts but also in appreciating NoSQL databases for 

In [17]:
query3 = evaluation_queries[3]
response = rag_pipeline(query3, collection_name, new_embedding_model)
print("Odpowiedź na pytanie:", response)

model='phi3:3.8b' created_at='2025-01-17T20:08:14.3907624Z' done=True done_reason='stop' total_duration=1497244208900 load_duration=20947900 prompt_eval_count=735 prompt_eval_duration=39515000000 eval_count=6091 eval_duration=1457684000000 message=Message(role='assistant', content='Dobra noc! W moim interesie i w pracy, ma masz doswiatok umiejętności techniczne oraz kontekstowej experymentacji z różnymi językami programowania. Mamy doświadczenie zarabiającą modeli predykcyjne, rozwiązując problem procesów medycznych w organizacjach osoby zdrowotnej i rodzinnego życia.\n\nPrawdziwym interesem jest automatyzacja komunikatu z państwami potocznymi oraz głosem społecznym, gdy trwa niepowodzenie leczeniowe. Takie technologii mogą posługiwałszyć osobie zdrowotnej w kluczowej dla swojego życia sytuacji i zminimalizowali konsekwencje nieprawidłowe leczenia.\n\nMoja praca zespołowa skomplikowana zawiera elementy zarządzania czasem, współpraca zespołową oraz przeze miasto wśród wielu miejscach, j

In [18]:
query4 = evaluation_queries[4]
response = rag_pipeline(query4, collection_name, new_embedding_model)
print("Odpowiedź na pytanie:", response)

model='phi3:3.8b' created_at='2025-01-17T20:14:31.2167617Z' done=True done_reason='stop' total_duration=374612386500 load_duration=131875300 prompt_eval_count=741 prompt_eval_duration=64667000000 eval_count=1448 eval_duration=309771000000 message=Message(role='assistant', content='\nGiven that Python Programming (40 marks)\n\nCraft a comprehensive and detailed critique of this document on "The Future of Artificial Neuroeconomics, Inc." within the context of social studies curriculum. The company wants to create an interactive fiction game where each day I will ask you for guidance as if we are working with two pets (a dog named Cezary in a fictitde city-state that was once ruled by ancient Babylon but now requires no more than 50 words, and the summary of your response must include:\n\n(i) A detailed breakdown of how to develop an optimal strategy for using social media marketing tactics. We are preparing a research project on \'Inkubating Researcher\'s Guidebook,\' I need help creatin

Odpowiedzi modelu llm (ChatGPT):
1. Jakie projekty związane z analizą danych realizowałeś w przeszłości?
W przeszłości realizowałam kilka projektów związanych z analizą danych, takich jak:

Nurses Schedule (2022/2023): Aplikacja w Pythonie generująca miesięczne harmonogramy dla pielęgniarek. Projekt wymagał analizy danych wejściowych, uwzględnienia preferencji użytkowników oraz generowania czytelnych raportów.
Modele predykcyjne rozwoju choroby serca (2023/2024): Praca inżynierska, w której analizowałam dane medyczne i budowałam modele predykcyjne, np. lasy losowe i regresję logistyczną.
Przewidywanie cen mieszkań w Bangalore (2024): Projekt w języku R, gdzie analizowałam dane rynkowe i budowałam modele przewidujące ceny nieruchomości.

2. Jakie technologie i narzędzia najczęściej wykorzystywałeś w swojej pracy?
Najczęściej korzystałam z:

Języków programowania: Python, R, SQL, C/C++.
Bazy danych: Microsoft SQL Server, obsługa zapytań SQL i optymalizacja wydajności.
Narzędzi analitycznych i wizualizacyjnych: Tableau, Power BI.
Narzędzi automatyzacji: Jenkins (do CI/CD), GitHub (zarządzanie wersjami).
Bibliotek Pythonowych: Pandas, NumPy, Matplotlib, Scikit-learn (analiza danych i modele uczenia maszynowego).

3. Czy możesz opisać swoje doświadczenie w pracy z bazami danych SQL?
Moje doświadczenie z SQL obejmuje:

Tworzenie i optymalizację zapytań SQL do analizy i przetwarzania danych.
Zarządzanie bazami danych w Microsoft SQL Server, w tym ich projektowanie i utrzymanie.
Analizę dużych zbiorów danych na potrzeby projektów takich jak modele predykcyjne chorób serca oraz przewidywanie cen mieszkań.
Automatyzację procesów związanych z przetwarzaniem danych w środowiskach SQL.

4. Jakie masz doświadczenie w optymalizacji procesów ETL?
W mojej pracy zajmowałam się automatyzacją i optymalizacją procesów ETL, co obejmowało:

Wykorzystanie narzędzi takich jak Jenkins do zautomatyzowanego przetwarzania danych.
Pisanie skryptów w Pythonie do transformacji i czyszczenia danych przed załadowaniem ich do baz danych.
Optymalizację przepływu danych i poprawę wydajności procesów ETL w projektach analitycznych.

5. Które osiągnięcie w Twojej karierze uważasz za najważniejsze?
Za najważniejsze osiągnięcie w mojej karierze uważam stworzenie modeli predykcyjnych rozwoju choroby serca w mojej pracy dyplomowej. Projekt ten wymagał nie tylko zaawansowanej analizy danych i wykorzystania modeli uczenia maszynowego, ale również dostosowania wyników do potrzeb praktycznych, takich jak ich interpretowalność dla lekarzy. Dzięki temu udało się połączyć wiedzę techniczną z realnym zastosowaniem w obszarze zdrowia publicznego.

## Pytania

Questions (2 points):

1. How does RAG improve the quality and reliability of LLM responses compared to pure LLM generation?
RAG poprawia odpowiedzi LLM, włączając zewnętrzne źródła informacji, co pozwala na generowanie dokładniejszych, bardziej trafnych i aktualnych odpowiedzi, opartych na danych zewnętrznych w czasie rzeczywistym. W przeciwieństwie do czystej generacji LLM, która polega wyłącznie na wewnętrznej wiedzy, RAG wzbogaca odpowiedzi o konkretne, kontekstowe informacje pobrane z zewnętrznych źródeł.

2. What are the key factors affecting RAG performance (chunk size, embedding quality, prompt design)?
Rozmiar fragmentu wpływa na ilość dostępnych informacji w każdym segmencie dokumentu, balansując między szczegółowością a kontekstem. Wysokiej jakości embeddingi zapewniają dokładność semantycznego wyszukiwania, natomiast dobrze zaprojektowane zapytania poprawiają jasność i celowość odpowiedzi modelu, kierując go precyzyjnym kontekstem.

3. How does the choice of vector database and embedding model impact system performance?
Baza danych wektorowa decyduje o szybkości i skalowalności procesu pobierania dokumentów, a zoptymalizowane bazy danych, takie jak FAISS czy Qdrant, oferują wydajniejsze możliwości wyszukiwania. Model embeddingu wpływa na trafność i dokładność pobranych dokumentów, ponieważ lepsze modele wychwytują bogatsze i bardziej precyzyjne informacje semantyczne, które pasują do zapytania.

4. What are the main challenges in implementing a production-ready RAG system?
Wyzwania obejmują zapewnienie skalowalności przy dużych zbiorach danych, utrzymanie dokładności i świeżości danych oraz optymalizację szybkości wyszukiwania przy dużym wolumenie zapytań. Dodatkowo integracja RAG z istniejącymi systemami i jego dostosowanie do obsługi złożonych i różnorodnych zapytań może wymagać dużych nakładów pracy.

5. How can the system be improved to handle complex queries requiring multiple document lookups?
Jak można ulepszyć system, aby obsługiwał złożone zapytania wymagające wielu wyszukiwań dokumentów?
Ulepszenia mogą obejmować zaprojektowanie systemu, który wspiera iteracyjne zapytania, gdzie złożone zapytania dzielone są na mniejsze podzapytania, które pobierają odpowiednie dokumenty etapami. Dodatkowo poprawienie strategii wyszukiwania o ranking kontekstowy i techniki łączenia wielu dokumentów pomoże w obsłudze bardziej złożonych typów zapytań.